In [3]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('key')
os.environ["KAGGLE_USERNAME"] = userdata.get('username')

In [2]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
License(s): CC0-1.0
 94% 214M/228M [00:01<00:00, 134MB/s]
100% 228M/228M [00:01<00:00, 135MB/s]


In [3]:
!mkdir data

In [4]:
!unzip /content/the-movies-dataset.zip -d /content/data

Archive:  /content/the-movies-dataset.zip
  inflating: /content/data/credits.csv  
  inflating: /content/data/keywords.csv  
  inflating: /content/data/links.csv  
  inflating: /content/data/links_small.csv  
  inflating: /content/data/movies_metadata.csv  
  inflating: /content/data/ratings.csv  
  inflating: /content/data/ratings_small.csv  


In [23]:
import pandas as pd
import numpy as np

In [24]:
df_meta=pd.read_csv('/content/data/movies_metadata.csv')
df_rating=pd.read_csv('/content/data/ratings.csv')

<ipython-input-24-cc698849f80c>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta=pd.read_csv('/content/data/movies_metadata.csv')


In [7]:
df_meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [16]:
df_meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [8]:
df_rating.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


#**Demographic Filtering -**

we need a metric to score or rate a movie
Calculate the score for every movie
Sort the scores and recommend the best rated movie to the users.

In [9]:
C= df_meta['vote_average'].mean()
print(C)

5.618207215134185


In [10]:
m= df_meta['vote_count'].quantile(0.9)
print(m)

160.0


In [11]:
q_movies = df_meta.copy().loc[df_meta['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [12]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [13]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [14]:
q_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,7.640253
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,6.820293
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5.660700
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,7.537201
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0,5.556626


In [15]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


#**Content Based Filtering**


In this recommender system the content of the movie (overview, cast, crew, keyword, tagline etc) is used to find its similarity with other movies. Then the movies that are most likely to be similar are recommended.

Plot description based Recommender
We will compute pairwise similarity scores for all movies based on their plot descriptions and recommend movies based on that similarity score. The plot description is given in the overview feature of our dataset. Let's take a look at the data.

In [17]:
!pip install gensim

In [18]:
import nltk
from nltk.corpus import stopwords
import gensim.downloader
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
def preprocessing(docs):
  docsAfter=[]
  for doc in docs:
    if type("string") == type(doc):
        doc = re.sub("[^A-Za-z0-9\s]", "", doc).split()
        doc=[word.lower() for word in doc if word not in stopwords.words('english')]
    docsAfter.append(doc)
  return docsAfter
def Doc2Vec(doc,Word2Vec):
  docVs=[]
  if type([])==type(doc) and len(doc)>0:
    for word in doc:
      if Word2Vec.has_index_for(word):
        docVs.append(Word2Vec.get_vector(word))
      else:
        docVs.append(np.zeros(Word2Vec.vector_size,))
  else:
    docVs.append(np.zeros((Word2Vec.vector_size,)))
    docVs.append(np.zeros((Word2Vec.vector_size,)))

  return  np.mean(np.array(docVs), axis=0)

def transformDocs2vecs(docs,model):
  docs=preprocessing(docs)
  docsVec= [Doc2Vec(doc,model) for doc in docs ]
  return docsVec

In [20]:
def mostSimilar(movie,docsVec,model,Idx2Movie,Movie2Idx,top=5):
  idx=Movie2Idx[movie]
  sorts=np.argsort(-model.cosine_similarities(docsVec[idx],docsVec[:]))[1:top]
  return [Idx2Movie[i] for i in sorts]

In [21]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [42]:
words_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [43]:
print(df_meta['overview'].head(5))
movies=df_meta['title']
docs=df_meta['overview'].values.tolist()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object


In [44]:
Idx2Movie=movies.to_dict()
Movie2Idx = dict((v,k) for k,v in Idx2Movie.items())

In [45]:
docsVec= transformDocs2vecs(docs,words_vectors)

**try recommendation system**

In [46]:
mostSimilar("Iron Man",docsVec,words_vectors,Idx2Movie,Movie2Idx,top=10)

['Longshot',
 'Young Again',
 'Red Line 7000',
 'Mr. Fix It',
 'Pillow to Post',
 'No Small Affair',
 'Elizabethtown',
 'Two-Faced Woman',
 'In This Our Life']

In [47]:
mostSimilar("Se7en",docsVec,words_vectors,Idx2Movie,Movie2Idx,top=10)

['The River Murders',
 'Baishe Srabon',
 'Beck 04 - Öga för öga',
 'I Saw the Devil',
 'The Unjust',
 'Talking to Heaven',
 'Strangled',
 'The Confessions of Thomas Quick',
 'Confession of Murder']

In [48]:
mostSimilar("The Dark Knight Rises",docsVec,words_vectors,Idx2Movie,Movie2Idx,top=10)

['The Dark Knight',
 'Batman: The Dark Knight Returns, Part 1',
 'Batman Returns',
 'Batman: The Dark Knight Returns, Part 2',
 'Batman',
 'Fantômas: Fantômas Against Fantômas',
 'Dillinger',
 'American Violet',
 'Batman Begins']

In [49]:
mostSimilar("Spectre",docsVec,words_vectors,Idx2Movie,Movie2Idx,top=10)

['Jack Reacher: Never Go Back',
 "Smiley's People",
 'Among Friends',
 'Hidden Agenda',
 'Madras Cafe',
 'Skyfall',
 'The Unjust',
 'The Interceptor',
 'Shin Godzilla']

In [50]:
mostSimilar("Mission: Impossible",docsVec,words_vectors,Idx2Movie,Movie2Idx,top=10)

['Hopscotch',
 'The Unjust',
 "Get Smart's Bruce and Lloyd Out of Control",
 'Little Nikita',
 '616: Paranormal Incident',
 'Mandroid',
 'Unthinkable',
 'The Point Men',
 'Reclaim']

In [51]:
mostSimilar("The Godfather",docsVec,words_vectors,Idx2Movie,Movie2Idx,top=10)

['The Sicilian Girl',
 'The Last Godfather',
 'Kill and Pray',
 'The Chamber',
 'Big Bad Mama II',
 'Karate Girl',
 'Fudoh: The New Generation',
 'Behind the Sun',
 'The Godfather: Part II']

#**Collaborative Filtering**

Collaborative Filtering (CF) recommends items to users based on their preferences and the preferences of similar users. Singular Value Decomposition (SVD) helps improve CF by breaking down a large user-item matrix into smaller parts, capturing key patterns and ignoring noise. By keeping only the most important patterns, SVD reduces the data's size and complexity. The reconstructed matrix from these parts can predict missing values, helping to recommend items users might like. SVD-based CF is effective because it handles large, sparse datasets well and focuses on the most relevant information for making accurate recommendations.

In [25]:
df_rating.head(10)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


In [26]:
df_rating.drop(columns='timestamp',inplace=True)

In [27]:
print(df_rating.shape)

(26024289, 3)


In [28]:
df_rating_count=df_rating.groupby("userId").count()

In [29]:
counter=1500

In [30]:
print(df_rating_count[df_rating_count['movieId']>counter].shape)
df_rating_count[df_rating_count['movieId']>counter].head()

(822, 2)


,movieId,rating
userId,,
741,2811,2811
836,1521,1521
1932,1767,1767
2520,1784,1784
2531,1928,1928


In [31]:
users_id=df_rating_count[df_rating_count['movieId']>counter].index.values
print(users_id.shape)

(822,)


In [32]:
df_rating_af=df_rating[df_rating['userId'].isin(users_id)]
df_rating_af.head()

,userId,movieId,rating
69308,741,1,4.5
69309,741,2,3.0
69310,741,3,3.5
69311,741,5,3.5
69312,741,6,3.5


In [33]:
df_rating_af.dtypes

userId       int64
movieId      int64
rating     float64
dtype: object

In [34]:
#df_rating_af=df_rating_af.astype({'userId': 'uint32','movieId': 'uint32','rating': 'float16'})

In [35]:
#df_rating_af.dtypes

In [36]:
#df_rating_af.head()

In [37]:
reviewmatrix = df_rating_af.pivot(index="userId", columns="movieId", values="rating").fillna(0)
reviewmatrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,176231,176233,176237,176239,176245,176257,176259,176263,176267,176273
userId,,,,,,,,,,,,,,,,,,,,,
741,4.5,3.0,3.5,0.0,3.5,3.5,0.0,2.5,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
836,3.5,3.5,3.5,0.0,0.0,0.0,3.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1932,3.5,4.0,1.5,0.0,1.0,4.0,1.5,0.0,2.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2520,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2531,3.5,3.0,2.0,0.0,0.0,4.5,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
from scipy.sparse.linalg import svds
matrix=reviewmatrix.values
matrix_mean = np.mean(matrix, axis=0)
matrix_demeaned = matrix - matrix_mean.T
k = 500  # Number of latent factors
users, sigma, movies = svds(matrix_demeaned, k)

In [51]:
moviesVec=movies.T

In [52]:
moviesVec.shape

(39604, 500)

In [53]:
def mostSimilar(movie,docsVec,Idx2Movie,Movie2Idx,top=5):
  idx=Movie2Idx[movie]
  sorts=np.argsort(-(np.dot(docsVec,docsVec[idx])/(np.linalg.norm(docsVec[idx])*np.linalg.norm(docsVec, axis=1))))[1:top]
  return [Idx2Movie[i] for i in sorts]

In [54]:
Idx2Movie=df_meta['title'].to_dict()
Movie2Idx = dict((v,k) for k,v in Idx2Movie.items())

In [55]:
mostSimilar("The Dark Knight Rises",moviesVec,Idx2Movie,Movie2Idx,top=10)

['Eklavya: The Royal Guard',
 'I Killed My Mother',
 'Rare Exports: A Christmas Tale',
 'According to Greta',
 'Offspring',
 'Superheroes',
 'Door-to-Door Maniac',
 'Smoke on the Potato Fields',
 'Passage to Marseille']

In [56]:
mostSimilar("The Godfather",moviesVec,Idx2Movie,Movie2Idx,top=10)

['Rollercoaster',
 "Nick and Norah's Infinite Playlist",
 'Best of the Badmen',
 'The Milky Way',
 "Le maître d'école",
 'Young Bess',
 'Ruby Sparks',
 "Pharaoh's  Army",
 'The Intruder']

In [57]:
mostSimilar("Spectre",moviesVec,Idx2Movie,Movie2Idx,top=10)

['The Exonerated',
 'Nayak: The Real Hero',
 '1915',
 'The Heineken Kidnapping',
 'Cave of the Sharks',
 'Do You Believe?',
 'The Garden of Earthly Delights',
 'The Flying Machine',
 'Dostana']

#**hybrid**